In [1]:
import tushare as ts
import pandas as pd

In [2]:
pro = ts.pro_api()

In [3]:
li = pd.read_excel('../data/databg.xlsx')

In [4]:
#
def gettprq(tscode,knowdate):
    findout = knowdate
    findout2 = knowdate
    dftp = pro.suspend(ts_code=tscode, suspend_date='', resume_date='', fields='')
    for susdate in dftp['suspend_date']:
        finddate = int(susdate)
        kndate = int(knowdate)
        if(finddate >= kndate):
            findout = str(finddate)
            findout2 = str(finddate)
            continue
        else:
            if((int(findout2)-finddate)>4):
                 findout2 = str(finddate)
                 break
            else:
                findout2 = str(finddate)
                continue
    return findout,findout2

In [5]:
timelist1 = []
ii = 0
for i in li['ts_code']:
    d=li[li['ts_code']==i]['news_date1'][ii]  # 公告日期
    ii = ii+1
    tetime = d.strftime('%Y%m%d')
    tprq,tprq2 = gettprq(i,tetime)
    if(int(tetime) == int(tprq)):
        print(ii)
    timelist1.append([tprq,tprq2])

3
5
10
11
14
15
16
19
20
21
23
24
25
26
27
28
29
30
32
35
37
38
39
40
41
42
46
49
51
52


In [6]:
li[['tingpai','tingpai2']] = timelist1

In [7]:
for i in li[['ts_code']]:
    d=li[li['ts_code']==i]['news_date1']
    print(d,i)

Series([], Name: news_date1, dtype: datetime64[ns]) ts_code


In [7]:
li.to_excel('../data/data_h2.xlsx')